In [ ]:
class Guitar():
    def __init__(self,tuning=[40, 45, 50, 55, 59, 64], fretnum=20):
        self.tuning= tuning
        self.fretnum=fretnum
        def genfboard():
            boarddic={}
            stri=6
            for i in self.tuning:
                for j in range(self.fretnum+1):
                    boarddic[(stri,j)]=i+j
                stri-=1
            return boarddic
        self.fboard=genfboard()

class Hand():
    def __init__(self):
        self.f1=None
        self.f2=None
        self.f3=None
        self.f4=None

In [2]:
test = [
    ['C', [60, 62, 64, 65, 64, 62, 60]], 
    ['Am', [57, 60, 64, 62, 60, 57]],     
    ['G', [55, 57, 59, 60, 59, 57, 55]],  
    ['D', [62, 64, 66, 69, 67, 64, 62]],  
    ['Em', [52, 55, 59, 60, 59, 55, 52]], 
    ['F', [65, 67, 69, 70, 69, 67, 65]]   
]
test[0]

['C', [60, 62, 64, 65, 64, 62, 60]]

In [ ]:
g1=Guitar()
g1.fboard